# Notebook: Quick accessibility analysis using H3pandas

Chingiz Zhanarbaev, 03.2024

This notebook demonstrates use-case of [NxTransit](https://github.com/chingiztob/NxTransit). In this notebook, we will use the library to calculate mean travel time for different parts of Chelyabinsk, Russia using [H3 grid](https://uber.github.io/h3-py/intro.html).

In [1]:
import pandas as pd
import osmnx as ox
import geopandas as gpd
import h3pandas

import nxtransit as nt

## Initialize data

The first step is to initialize a graph object. Transit graph is a `nx.DiGraph` object which contains special attributes for handling dynamic nature of GTFS data.

In [2]:
# Setting up the parameters
GTFSpath = "Feeds\Chelyabinsk" # Path to the folder with GTFS files
departure_time_input = "12:00:00" # # Departure time in HH:MM:SS format
day = 'monday' # # Day of the week

In [3]:
transit_graph = nt.feed_to_graph(
    GTFSpath=GTFSpath,
    day_of_week=day,
    departure_time_input=departure_time_input,
    duration_seconds=3600 * 14,
    multiprocessing=True,
)

GTFS feed is valid.
151276 of 242644 trips retained
Building graph in parallel


INFO - Transit graph created
INFO - Loading OSM graph via OSMNX
INFO - Street network graph created
INFO - Nodes: 87834, Edges: 239108


### Prepare buildings geodataframe using OSMnx

In [ ]:
tags = {"building": True}
buildings = ox.features_from_place("Chelyabinsk", tags)
building_centroids = gpd.GeoDataFrame(buildings, geometry=buildings.geometry.centroid)

### Aggregate OSM buildings with H3 cells using `h3pandas`

Resulting geodataframe will contain only cells that have buildings in them.

In [5]:
building_centroids_h3 = building_centroids.h3.geo_to_h3_aggregate(9, return_geometry=True)
# Filter unnecessary columns
building_centroids_h3 = building_centroids_h3[["geometry"]]

In [6]:
building_centroids_h3.explore(tooltip="h3_09", tiles = "cartodbpositron")

## Calculate OD-cost matrix

### Snap H3 centroids to network

Add `id` field to the polygons, which will be used as a node id in the graph

In [7]:
building_centroids_h3['id'] = building_centroids_h3.index
hex_centroids = nt.create_centroids_dataframe(building_centroids_h3)
nt.snap_points_to_network(transit_graph, hex_centroids)

### Perform OD-cost matrix calculation

In [8]:
# Create list of nodes for OD calculation
nodes = list(hex_centroids["origin_id"])
od_matrix = nt.calculate_od_matrix_parallel(graph=transit_graph, nodes=nodes, departure_time=3600 * 18, num_processes=6)

Calculating the OD using 6 processes
Time elapsed: 567.5380474000121


Group OD-cost matrix by source node and calculate the average travel time to all other nodes

In [9]:
od_group = od_matrix.groupby('source_node')['travel_time'].mean().reset_index()

Merge data from OD-cost matrix with the H3 geodataframe

In [10]:
df = pd.merge(building_centroids_h3, od_group, left_on="id", right_on="source_node")
df = df[["geometry", "travel_time"]]

## Plot results using folium

In [11]:
df.explore(
    column="travel_time",
    cmap="RdYlGn_r",
    tiles = "cartodbpositron",
    legend=True,
    tooltip="travel_time",
    legend_kwds=dict(colorbar=False),
    scheme="Quantiles",
    k=6,
)